In [1]:
from langgraph.graph import StateGraph,START, END
from typing import TypedDict
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI

load_dotenv()

/Users/priyajhanji/Documents/personal-git-abhikaura25/ai-engineering-hub/venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
None of PyTorch, TensorFlow >= 2.0, or Flax have been found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.


True

In [2]:
class BatsmenStatisticsState(TypedDict):
    runs: int
    balls_faced: int
    number_of_fours: int
    number_of_sixes: int
    strike_rate: float
    boundary_percentage: float
    balls_per_boundary: float

In [5]:
def calculate_strike_rate(state: BatsmenStatisticsState) -> dict:
    strike_rate = (state['runs'] / state['balls_faced']) * 100
    return {'strike_rate': strike_rate}

def calculate_boundary_percentage(state: BatsmenStatisticsState) -> dict:
    total_boundaries = state['number_of_fours'] + state['number_of_sixes']
    boundary_percentage = (total_boundaries / state['balls_faced']) * 100
    return {'boundary_percentage': boundary_percentage}

def balls_per_boundary(state: BatsmenStatisticsState) -> dict:
    total_boundaries = state['number_of_fours'] + state['number_of_sixes']
    if total_boundaries > 0:
        bpb = state['balls_faced'] / total_boundaries
    else:
        bpb = float('inf')  # No boundaries hit
    return {'balls_per_boundary': bpb}

In [8]:
graph = StateGraph(BatsmenStatisticsState)

graph.add_node("calculate_strike_rate", calculate_strike_rate)
graph.add_node("calculate_boundary_percentage", calculate_boundary_percentage)
graph.add_node("balls_per_boundary", balls_per_boundary)

graph.add_edge(START, "calculate_strike_rate")
graph.add_edge(START, "calculate_boundary_percentage")
graph.add_edge(START, "balls_per_boundary")

graph.add_edge("calculate_strike_rate", END)
graph.add_edge("calculate_boundary_percentage", END)
graph.add_edge("balls_per_boundary", END)   

compiled_graph = graph.compile()

result = compiled_graph.invoke({
    "runs": 85,
    "balls_faced": 60,
    "number_of_fours": 8,
    "number_of_sixes": 2
}) 

print(result)

{'runs': 85, 'balls_faced': 60, 'number_of_fours': 8, 'number_of_sixes': 2, 'strike_rate': 141.66666666666669, 'boundary_percentage': 16.666666666666664, 'balls_per_boundary': 6.0}
